# Proyecto
### Carrela Silva Carlos Francisco Javier

## Importar librerias

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
# Statistics
from scipy.stats import ks_2samp
from scipy.stats import chisquare
#visualization
import cufflinks as cf
#reduccion de variables
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split # pip install scikit-learn
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
# from varclushi import VarClusHi
import matplotlib.pyplot as plt

## La intencion de nuestro trabajo sera crear un modelo que nos ayude a calcular la apuesta Over de 2 goles de un determinado partido

## Lectura de datos

In [ ]:
club_games=pd.read_csv("./club_games.csv")
clubs=pd.read_csv("./clubs.csv")
game_events=pd.read_csv("./game_events.csv")
games=pd.read_csv("./games.csv")

In [ ]:
club_games.sample(5)

In [ ]:
clubs.sample(5)

In [ ]:
game_events.sample(5)

In [ ]:
games.sample(5)

## Agrupar y unir las tablas


In [ ]:
grouped_game_goals= game_events.groupby(["game_id"]).agg({'type': lambda x: sum(x == 'Goals')}).sort_values('game_id')#esta columna nos dira cuantas Goles hubo por partido
grouped_game_substitutions= game_events.groupby(["game_id"]).agg({'type': lambda x: sum(x == 'Substitutions')}).sort_values('game_id')#esta columna nos dira cuantas sustituciones hubo por partido
goals_addedtime=game_events.groupby(["game_id"]).agg({'minute': lambda x: sum(x >90)}).sort_values('game_id') #esta columna nos dice el numero de goles en tiempo de compensacion
minute_last_goal = game_events.groupby('game_id').apply(lambda x: x.loc[x['type'] == 'Goals', 'minute'].max()).fillna(0).sort_index().reset_index()#minuto del ultimo gol
minute_first_goal = game_events.groupby('game_id').apply(lambda x: x.loc[x['type'] == 'Goals', 'minute'].min()).fillna(0).sort_index().reset_index() #minuto del primero gol

In [ ]:
events=grouped_game_goals
events["number_game_substitutions"]=grouped_game_substitutions["type"]
events["goals_added_time"]=goals_addedtime["minute"]
events["minute_last_goal"]=minute_last_goal.set_index("game_id")[0]
events["minute_first_goal"]=minute_first_goal.set_index("game_id")[0]
events = events.rename(columns={'type': 'number_goals'})
events



In [ ]:
events.describe()


In [ ]:
club_games # de esta tabla podemos conservar todo si solo tomamos como referencia a los partidos de locales, de todas formas se incluiran los datos de todos los partidos

In [ ]:

#la tabla games esta lista para usar
games

In [ ]:
merge_table= pd.merge(games.set_index("game_id"), club_games.set_index("game_id"), left_index=True, right_index=True,how="right")
merge_table=pd.merge(merge_table, events, left_index=True, right_index=True,how="inner")


In [ ]:
# para agregar los datos de el club local y visitante renombrare los datos para poder identificarlos, creando 2 nuevas tablas con los nombres
local=clubs
visit=clubs

for column in local.columns:
    local=local.rename(columns={column: f"{column}_local"})
    visit=visit.rename(columns={column: f"{column}_visit"})




In [ ]:
#agregamos a la tabla los datos tanto del club local como el visitante
merge_table=pd.merge(merge_table.set_index("home_club_id"), local.set_index("club_id_local"), left_index=True, right_index=True,how="left")
merge_table=pd.merge(merge_table.set_index("away_club_id"), visit.set_index("club_id_visit"), left_index=True, right_index=True,how="left")


In [ ]:
# vamos a hacer que la columna de fecha se reconozca de tal forma para que podamos ordenrar todo por fecha
merge_table['date'] = pd.to_datetime(merge_table['date'])
merge_table.info()

In [ ]:
# vamos a orderar nuestro df por fecha
merge_table= merge_table.sort_values('date')
merge_table=merge_table.reset_index()
merge_table

In [ ]:
merge_table.columns

## Ingieneria de variables

In [ ]:
#como lo que queremos predecir son los resultados de los partidos ,vamos a desplazar las viariables que tienen que ver con
#el resultado del partido

#para el partido pasado del local
grouped = merge_table.groupby('club_id')
merge_table['casa_goles_anteriores'] = grouped['home_club_goals'].shift()
merge_table['casa_goles_anteriores_recibidos'] = grouped['away_club_goals'].shift()
merge_table['casa_minuto_primer_gol_anterior'] = grouped['minute_first_goal'].shift()
merge_table['casa_minuto_ultimo_gol_anterior'] = grouped['minute_last_goal'].shift()
merge_table['casa_victoria_local_partidoanterior'] = grouped['is_win'].shift()
merge_table['casa_number_goals_anterior'] = grouped['number_goals'].shift()
merge_table['casa_sustituciones_anterior'] = grouped['number_game_substitutions'].shift()
merge_table['casa_goles_tiempo_añadido_anterior'] = grouped['goals_added_time'].shift()

#para el partido pasados del visitante
grouped = merge_table.groupby('opponent_id')
merge_table['visita_goles_anteriores_recibidos'] = grouped['home_club_goals'].shift()
merge_table['visita_goles_anteriores'] = grouped['away_club_goals'].shift()
merge_table['visita_minuto_primer_gol_anterior'] = grouped['minute_first_goal'].shift()
merge_table['visita_minuto_ultimo_gol_anterior'] = grouped['minute_last_goal'].shift()
merge_table['visita_victoria_local_partidoanterior'] = grouped['is_win'].shift()*-1
merge_table['visita_number_goals_anterior'] = grouped['number_goals'].shift()
merge_table['visita_sustituciones_anterior'] = grouped['number_game_substitutions'].shift()
merge_table['visita_goles_tiempo_añadido_anterior'] = grouped['goals_added_time'].shift()
merge_table.info()

In [ ]:
#como lo que queremos predecir son los resultados de los partidos en realida,vamos a desplazar las viariables que tienen que ver con
#el resultado del partido ahora dando info de los 2 partidos previos

#para el partido pasado del local
grouped = merge_table.groupby('club_id')
merge_table['casa_goles_anteriores_2'] = grouped['home_club_goals'].shift(2)
merge_table['casa_goles_anteriores_recibidos_2'] = grouped['away_club_goals'].shift(2)
merge_table['casa_minuto_primer_gol_anterior_2'] = grouped['minute_first_goal'].shift(2)
merge_table['casa_minuto_ultimo_gol_anterior_2'] = grouped['minute_last_goal'].shift(2)
merge_table['casa_victoria_local_partidoanterior_2'] = grouped['is_win'].shift(2)
merge_table['casa_number_goals_anterior_2'] = grouped['number_goals'].shift(2)
merge_table['casa_sustituciones_anterior_2'] = grouped['number_game_substitutions'].shift(2)
merge_table['casa_goles_tiempo_añadido_anterior_2'] = grouped['goals_added_time'].shift(2)

#para el partido pasados del visitante
grouped = merge_table.groupby('opponent_id')
merge_table['visita_goles_anteriores_recibidos_2'] = grouped['home_club_goals'].shift(2)
merge_table['visita_goles_anteriores_2'] = grouped['away_club_goals'].shift(2)
merge_table['visita_minuto_primer_gol_anterior_2'] = grouped['minute_first_goal'].shift(2)
merge_table['visita_minuto_ultimo_gol_anterior_2'] = grouped['minute_last_goal'].shift(2)
merge_table['visita_victoria_local_partidoanterior_2'] = grouped['is_win'].shift(2)*-1
merge_table['visita_number_goals_anterior_2'] = grouped['number_goals'].shift(2)
merge_table['visita_sustituciones_anterior_2'] = grouped['number_game_substitutions'].shift(2)
merge_table['visita_goles_tiempo_añadido_anterior_2'] = grouped['goals_added_time'].shift(2)
merge_table.info()

In [ ]:
#como lo que queremos predecir son los resultados de los partidos en realida,vamos a desplazar las viariables que tienen que ver con
#el resultado del partido ahora dando info de los 3 partidos previos

#para el partido pasado del local
grouped = merge_table.groupby('club_id')
merge_table['casa_goles_anteriores_3'] = grouped['home_club_goals'].shift(3)
merge_table['casa_goles_anteriores_recibidos_3'] = grouped['away_club_goals'].shift(3)
merge_table['casa_minuto_primer_gol_anterior_3'] = grouped['minute_first_goal'].shift(3)
merge_table['casa_minuto_ultimo_gol_anterior_3'] = grouped['minute_last_goal'].shift(3)
merge_table['casa_victoria_local_partidoanterior_3'] = grouped['is_win'].shift(3)
merge_table['casa_number_goals_anterior_3'] = grouped['number_goals'].shift(3)
merge_table['casa_sustituciones_anterior_3'] = grouped['number_game_substitutions'].shift(3)
merge_table['casa_goles_tiempo_añadido_anterior_3'] = grouped['goals_added_time'].shift(3)

#para el partido pasados del visitante
grouped = merge_table.groupby('opponent_id')
merge_table['visita_goles_anteriores_recibidos_3'] = grouped['home_club_goals'].shift(3)
merge_table['visita_goles_anteriores_3'] = grouped['away_club_goals'].shift(3)
merge_table['visita_minuto_primer_gol_anterior_3'] = grouped['minute_first_goal'].shift(3)
merge_table['visita_minuto_ultimo_gol_anterior_3'] = grouped['minute_last_goal'].shift(3)
merge_table['visita_victoria_local_partidoanterior_3'] = grouped['is_win'].shift(3)*-1
merge_table['visita_number_goals_anterior_3'] = grouped['number_goals'].shift(3)
merge_table['visita_sustituciones_anterior_3'] = grouped['number_game_substitutions'].shift(3)
merge_table['visita_goles_tiempo_añadido_anterior_3'] = grouped['goals_added_time'].shift(3)
merge_table.info()

In [ ]:
# siguiendo el mismo pensamiento de que queremos predecir el total de goles, antes de un partido
#eleminaremos las variables correspondientes alos sucesos ocurridos durante el partido, ya que no tiene sentido
#querer predecir total de goles, si el partido ya termino, a excepcion de nuestra variable objetivo
#tambien eliminaremos variables que no tienen sentido, como tamaño de estadio de la visita, ya que es un factor totoalmente ajeno
#y variables que venian repetidas por dos tablas
ls_delte=['stadium_seats_visit',"home_club_goals","away_club_goals","minute_first_goal","minute_last_goal",'is_win','aggregate','own_goals','opponent_goals' ,'number_game_substitutions','goals_added_time',"own_manager_name","opponent_manager_name"]

for i in ls_delte:
    merge_table=merge_table.drop([i],axis=1)

merge_table.sample(5)





In [ ]:

merge_table=merge_table.sort_values('date')


In [ ]:
# creacion variable objetivo
merge_table["over_2"]=merge_table["number_goals"].map(lambda x: 1 if x > 2 else 0)

In [ ]:
# Veamos cuales son las variables continuas para crear mas variables a partir de ellas
ls_cont=[]
for i in  merge_table.columns:
        if not  merge_table[i].dtype == object:
            if merge_table[i].nunique() > 2:
                ls_cont.append(i)

In [ ]:
merge_table[ls_cont].columns

In [ ]:
# Calcular el promedio actualizado de goles recibidos por equipo utilizando rolling y groupby

merge_table["casa_goles_recibidos_mean"] = merge_table.groupby('club_id')['casa_goles_anteriores_recibidos'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
merge_table["casa_goles_anotados_mean"]= merge_table.groupby('club_id')['casa_goles_anteriores'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
merge_table["visita_goles_recibidos_mean"] = merge_table.groupby('opponent_id')['visita_goles_anteriores_recibidos'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
merge_table["visita_goles_anotados_mean"]= merge_table.groupby('opponent_id')['visita_goles_anteriores'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
merge_table["casa_number_goals_mean"]= merge_table.groupby('opponent_id')['casa_number_goals_anterior'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)
merge_table["visita_number_goals_mean"]= merge_table.groupby('opponent_id')['visita_number_goals_anterior'].rolling(window=5, min_periods=1).mean().reset_index(drop=True)

In [ ]:
# Creacion de variables continuas para 1 partido anterior
merge_table["age_difference"]=merge_table['average_age_local']-merge_table['average_age_visit']
merge_table["foreigners_percentage_difference"]=merge_table['foreigners_percentage_local']-merge_table['foreigners_percentage_visit']
merge_table["suma_goles_recibidos"]=merge_table['casa_goles_anteriores_recibidos']+merge_table['visita_goles_anteriores_recibidos']
merge_table["suma_goles_anotados"]=merge_table['casa_goles_anteriores']+merge_table['visita_goles_anteriores']
merge_table["total_de_goles_anterior_casa"]=merge_table['casa_goles_anteriores']+merge_table['casa_goles_anteriores_recibidos']
merge_table["total_de_goles_anterior_visita"]=merge_table['visita_goles_anteriores']+merge_table['visita_goles_anteriores_recibidos']
merge_table["anotado_local_recibido_visita"]=merge_table['casa_goles_anteriores']+merge_table['visita_goles_anteriores_recibidos']
merge_table["anotado_visita_recibido_local"]=merge_table['casa_goles_anteriores_recibidos']+merge_table['visita_goles_anteriores']
merge_table['over_2_anterior_casa']=merge_table["total_de_goles_anterior_casa"].map(lambda x: 1 if x > 2 else 0)
merge_table['over_2_anterior_visita']=merge_table["total_de_goles_anterior_visita"].map(lambda x: 1 if x > 2 else 0)
merge_table["diferencia_promedio_anotados"]=merge_table["casa_goles_anotados_mean"]-merge_table["visita_goles_anotados_mean"]
merge_table["diferencia_promedio_recibidos"]=merge_table["casa_goles_recibidos_mean"]-merge_table["visita_goles_recibidos_mean"]
merge_table["suma_promedio_recibidos_anotados"]=merge_table["casa_goles_recibidos_mean"]+merge_table["visita_goles_anotados_mean"]
merge_table["suma_promedio_anotados_recibidos"]=merge_table["casa_goles_anotados_mean"]+merge_table["visita_goles_recibidos_mean"]
merge_table["suma_promedio_number_goals"]=merge_table["casa_number_goals_mean"]+merge_table["visita_number_goals_mean"]
merge_table["diferencia_promedio_number_goals"]=merge_table["casa_number_goals_mean"]-merge_table["visita_number_goals_mean"]
merge_table["diferencia_sustituciones"]=merge_table['casa_sustituciones_anterior']-merge_table['visita_sustituciones_anterior']



In [ ]:
# Creacion de variables continuas para 2 partidos anteriores
merge_table["suma_goles_recibidos_2"]=merge_table['casa_goles_anteriores_recibidos_2']+merge_table['visita_goles_anteriores_recibidos_2']
merge_table["suma_goles_anotados_2"]=merge_table['casa_goles_anteriores_2']+merge_table['visita_goles_anteriores_2']
merge_table["total_de_goles_anterior_casa_2"]=merge_table['casa_goles_anteriores_2']+merge_table['casa_goles_anteriores_recibidos_2']
merge_table["total_de_goles_anterior_visita_2"]=merge_table['visita_goles_anteriores_2']+merge_table['visita_goles_anteriores_recibidos_2']
merge_table["anotado_local_recibido_visita_2"]=merge_table['casa_goles_anteriores_2']+merge_table['visita_goles_anteriores_recibidos_2']
merge_table["anotado_visita_recibido_local_2"]=merge_table['casa_goles_anteriores_recibidos_2']+merge_table['visita_goles_anteriores_2']
merge_table['over_2_anterior_casa_2']=merge_table["total_de_goles_anterior_casa_2"].map(lambda x: 1 if x > 2 else 0)
merge_table['over_2_anterior_visita_2']=merge_table["total_de_goles_anterior_visita_2"].map(lambda x: 1 if x > 2 else 0)
merge_table["diferencia_sustituciones_2"]=merge_table['casa_sustituciones_anterior_2']-merge_table['visita_sustituciones_anterior_2']

In [ ]:
# Creacion de variables continuas para 3 partidos anteriores
merge_table["suma_goles_recibidos_3"]=merge_table['casa_goles_anteriores_recibidos_3']+merge_table['visita_goles_anteriores_recibidos_3']
merge_table["suma_goles_anotados_3"]=merge_table['casa_goles_anteriores_3']+merge_table['visita_goles_anteriores_3']
merge_table["total_de_goles_anterior_casa_3"]=merge_table['casa_goles_anteriores_3']+merge_table['casa_goles_anteriores_recibidos_3']
merge_table["total_de_goles_anterior_visita_3"]=merge_table['visita_goles_anteriores_3']+merge_table['visita_goles_anteriores_recibidos_3']
merge_table["anotado_local_recibido_visita_3"]=merge_table['casa_goles_anteriores_3']+merge_table['visita_goles_anteriores_recibidos_3']
merge_table["anotado_visita_recibido_local_3"]=merge_table['casa_goles_anteriores_recibidos_3']+merge_table['visita_goles_anteriores_3']
merge_table['over_2_anterior_casa_3']=merge_table["total_de_goles_anterior_casa_3"].map(lambda x: 1 if x > 2 else 0)
merge_table['over_2_anterior_visita_3']=merge_table["total_de_goles_anterior_visita_3"].map(lambda x: 1 if x > 2 else 0)
merge_table["diferencia_sustituciones_3"]=merge_table['casa_sustituciones_anterior_3']-merge_table['visita_sustituciones_anterior_3']

In [ ]:
merge_table=merge_table.set_index('date')

# EDA

In [ ]:
# Distribucion de variables continuas
ls_cont=[]
for i in  merge_table.columns:
    if not  merge_table[i].dtype == object:
        if merge_table[i].nunique() > 2:
            res=merge_table[i].map(lambda x:float(x)).iplot(kind="hist", title=f"{i}'s histogram", asFigure=True)
            res.show()#descomentar para eda
            ls_cont.append(i)


In [ ]:
# Distrinucion de variables discretas
!pip install chart_studio
import plotly.express as px
ls_disc=[]
for i in merge_table.columns:
        if merge_table[i].nunique() < 30:
            if i in ls_cont:
                continue
            else:
                ls_disc.append(i)
for variable in ls_disc:
    fig = px.pie(merge_table[variable].value_counts().reset_index(), names='index', values=variable, title=f"{variable}'s distribution")
    fig.show()

In [ ]:
#  Se comento ya que el notebook pesaba demasiado, pero se incluye en el reporte
"""# Distribucion de variales en relacion con la variable objetivo
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.offline as pyo
for variable in ls_cont:
    data = []

    trace1 = go.Histogram(
        x=merge_table[merge_table['over_2'] == 0][variable],
        name='Categoría 0',
        histnorm='percent',
        opacity=0.5
    )
    data.append(trace1)

    trace2 = go.Histogram(
        x=merge_table[merge_table['over_2'] == 1][variable],
        name='Categoría 1',
        histnorm='percent',
        opacity=0.5
    )
    data.append(trace2)

    layout = go.Layout(
        title='Histograma de ' + variable + ' con respecto a la Categoría ' + 'over_2',
        xaxis=dict(title='Valor'),
        yaxis=dict(title='Frecuencia %'),
        barmode='overlay'
    )

    fig = go.Figure(data=data, layout=layout)
    pyo.iplot(fig)""" #descomentar para eda

In [ ]:
merge_table.info()

In [ ]:

ls_disc

In [ ]:
#eliminasmos la variable "hosting"
merge_table=merge_table.drop(["hosting"], axis=1)


In [ ]:
ls_disc.remove("hosting")

# Creamos dummies para las variables discretas tipo string

In [ ]:
# dummies de la variable subject
ls_dummies=[]
for i in ls_disc:
    if merge_table[i].dtype == object:
        ls_dummies.append(i)
dumm=pd.get_dummies(merge_table[ls_dummies],dtype=int,drop_first=True) # se usa tipo entero y se elimina una de las columnas para no repetir informacion
for i in dumm.columns: # se agregan las columnas al df original
    merge_table[i]=dumm[i]
merge_table.info()

In [ ]:
# actualizamos la lista de variables discretas
ls_disc=[]
for i in merge_table.columns:
    if i in ls_cont:
        continue
    else:
        if not merge_table[i].dtype == object:
            ls_disc.append(i)
len(ls_disc)

# Limpieza

In [ ]:
# vamos a quitar las variables que no nos interesan de ls_cont

ls_cont.remove('club_id')


In [ ]:
ls_cont.remove('opponent_id')

In [ ]:
#vemos que no tenemos valores unarios para este caso
merge_table[ls_cont+ls_disc].nunique().sort_values().to_frame() #vemos que ya no hay mas vairbales unarias

In [ ]:
nulos=merge_table[ls_disc+ls_cont].isnull().mean().sort_values().to_frame().reset_index()
nulos



In [ ]:
# actualizamos a nuestro df
merge_table=merge_table[ls_cont+ls_disc+['club_id','opponent_id']]

In [ ]:
# remocion de variables con valores ausentes
shape_df=merge_table.shape
ls_miss_var=[]
nulos=nulos[nulos[0]>.15]
for i in nulos["index"]:
        ls_miss_var.append(f"{i}")
print(f"nuestro conjunto de datos tiene {len(ls_miss_var)} variables con mas del 15% de ausentes y son {ls_miss_var}")

In [ ]:
#vamos a eliminar todas las columnas con mas del 10% de valores ausentes
ls_position=["home_club_position","away_club_position"]#notamos que estas varibles no tienen nulos, pero el 25 de los datos son -1, lo cual es falta de info
for i in ls_miss_var+ls_position:
    merge_table=merge_table.drop([i],axis=1)


In [ ]:
#ahora solo tenemos 34 columnas pero ninguna tiene mas del 10% de valores ausentes, sin embargo debemos de darle un tratamiento a los valores ausentes que tenemos
merge_table.info()

In [ ]:
# los demas valores nulos son resultado de que las variables que creamos,son apartir de otros registros anteriores, por lo que el primer registro
#de cada equipo como local y visita tendra valor nulo al no tener valor anterior, por lo que podemos desasernos de estos valores.
merge_table=merge_table.dropna()
merge_table.info()

In [ ]:
# vamos a actualizar la lista ls_cont y ls_disc
ls_cont=[]
for i in merge_table.columns:
    if merge_table[i].nunique()>2:
        ls_cont.append(i)
ls_disc=[]
for i in merge_table.columns:
    if not i in ls_cont:
        ls_disc.append(i)
print(ls_cont,len(ls_cont))
print(ls_disc, len(ls_disc))

In [ ]:
#vamos a remover las variables altamente correlacionadas
corr_matrix = merge_table[ls_cont].corr()
ls_checked = []
ls_correlated = []
for col in corr_matrix.columns:
    ls_checked.append(col)
    ls_correlated += corr_matrix[(corr_matrix[col] >= .90) & (~corr_matrix.index.isin(ls_checked))].index.tolist()
ls_correlated = list(set(ls_correlated))
ls_correlated = [variable for variable in ls_correlated if variable not in ls_disc ]
print(f"las variables con correlacion de 1 con respecto a otr son{ls_correlated}")
if len(ls_correlated) >1:
    merge_table.drop(ls_correlated,axis=1)

# deteccion de valores extremos

In [ ]:
#definimos funcion para detectar outliers

def detect_outlier(serie, method):
    if method == "iqr":
        q1 = serie.quantile(.25)
        q3 = serie.quantile(.75)
        iqr = q3-q1
        upper_fence = q3 + 1.5*iqr
        lower_fence = q1 - 1.5*iqr
    elif method == "z-score":
        mean = serie.mean()
        std = serie.std()
        upper_fence = mean + 3*std
        lower_fence = mean - 3*std
    else:
        upper_fence = serie.quantile(.99)
        lower_fence = serie.quantile(.01)
    return ~serie.between(lower_fence, upper_fence, inclusive="both")

In [ ]:
Xp = pd.concat(map(lambda column: detect_outlier(merge_table[column], "other").rename(f"{column}_ol"), ls_cont), axis=1)

In [ ]:
shape_old = merge_table.shape
shape_new=Xp[Xp.mean(axis=1)<0.3].shape #renglones que tengan menos del 30% de variables detectadas como outlier
shape_new[0] / shape_old[0] # proporcion de tamaño del nuevo data frame, sin valores extremos

In [ ]:
#podemos ver que con el metodo que menos perdemos registros, es con el z_core, para este caso nos quedaremos con
#los cuantiles
merge_table =merge_table[Xp.mean(axis=1)<0.3].reset_index(drop=True)
merge_table

In [ ]:
# variables con infinitos
#Validamos ahora para inf variables
inf_values = np.isinf(merge_table[ls_cont]).sum().items()
inf_variables = [var for var, n_inf in inf_values if n_inf > 0]
inf_variables # no hay variables con infinitos
if len(inf_variables)>1:
    merge_table.drop(inf_variables,axis=1)

# Separacion de conjuntos

In [ ]:
x=merge_table.copy().drop(['number_goals','over_2'],axis=1)
y=merge_table["over_2"]


In [ ]:
# juntamos los 3 conjuntos
proporcion_entrenamiento = 0.7
proporcion_validacion = 0.2
proporcion_prueba = 0.1

# Calcular el tamaño de cada conjunto
tamano_datos = len(x)
tamano_entrenamiento = int(proporcion_entrenamiento * tamano_datos)
tamano_validacion = int(proporcion_validacion * tamano_datos)

# Calcular los índices de división
indice_entrenamiento = tamano_entrenamiento
indice_validacion = tamano_entrenamiento + tamano_validacion

# Crear los conjuntos de datos
x_train = x[:indice_entrenamiento].drop(['club_id','opponent_id','index'],axis=1)
x_test = x[indice_entrenamiento:indice_validacion].drop(['club_id','opponent_id','index'],axis=1)
x_validate = x[indice_validacion:].drop(['club_id','opponent_id','index'],axis=1)
y_train = y[:indice_entrenamiento]
y_test = y[indice_entrenamiento:indice_validacion]
y_validate = y[indice_validacion:]
validate_local_id=x[:indice_entrenamiento]['club_id']
validate_visit_id=x[:indice_entrenamiento]['opponent_id']


# Escalamiento

In [ ]:
scaler=MinMaxScaler()
scaler.fit(x_train)
X_scaled=pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
X_test_scaled=pd.DataFrame(scaler.transform(x_test), columns=x_train.columns)
X_validate_scaled=pd.DataFrame(scaler.transform(x_validate), columns=x_train.columns)


In [ ]:
X_scaled


In [ ]:

# para guargar en la computadora
X_scaled.to_csv('xs.csv', index=False)
X_test_scaled.to_csv('xs_test.csv', index=False)
X_validate_scaled.to_csv('xs_validate.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
y_validate.to_csv('y_validate.csv', index=False)
validate_local_id.to_csv('validate_local_id.csv', index=False)
validate_visit_id.to_csv('validate_visit_id.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# para guardar en drive

In [ ]:
X_scaled.to_csv('/content/drive/My Drive/xs.csv', index=False)
X_test_scaled.to_csv('/content/drive/My Drive/xs_test.csv', index=False)
X_validate_scaled.to_csv('/content/drive/My Drive/xs_validate.csv', index=False)
y_train.to_csv('/content/drive/My Drive/y_train.csv', index=False)
y_test.to_csv('/content/drive/My Drive/y_test.csv', index=False)
y_validate.to_csv('/content/drive/My Drive/y_validate.csv', index=False)
validate_local_id.to_csv('/content/drive/My Drive/validate_local_id.csv', index=False)
validate_visit_id.to_csv('/content/drive/My Drive/validate_visit_id.csv', index=False)